In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks/konkurs'
!pip install --upgrade tables
import pandas as pd
import numpy as np
np.random.seed(0)

/content/drive/My Drive/Colab Notebooks/konkurs
Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [ ]:
# get data
train = pd.read_hdf('./train.car_price.h5')
#del train["breadcrumb"]
test = pd.read_hdf('./test.car_price.h5')
#del test["breadcrumb"]

In [ ]:
def change_eur_to_pln(row):
  if row['price_currency'] == 'PLN':
    return row
  elif row['price_currency'] == 'EUR':
    row['price_value'] = row['price_value']*4
    return row
  else:
    raise EnvironmentError

train = train.apply(change_eur_to_pln, axis=1)

In [ ]:
train['log_price'] = np.log(train["price_value"])

In [ ]:
all_data = pd.concat([train, test])

start here

In [ ]:
def find_marka(row):
  if row["param_marka-pojazdu"] is not None:
    return row["param_marka-pojazdu"]
  elif row["param_make"] is not None:
    return row["param_make"]
  else:
    return -1

all_data["my_marka"] = all_data.apply(find_marka, axis=1)
#del all_data["param_marka-pojazdu"]
#del all_data["param_make"]

In [ ]:
def find_model(row):
  if row["param_model-pojazdu"] is not None:
    return row["param_model-pojazdu"]
  elif row["param_model"] is not None:
    return row["param_model"]
  else:
    return -1

all_data["my_model"] = all_data.apply(find_model, axis=1)
#del all_data["param_model-pojazdu"]
#del all_data["param_model"]

In [ ]:
all_data["my_marka_factor"] = all_data["param_marka-pojazdu"].factorize()[0]
# del all_data["param_marka-pojazdu"]

In [ ]:
def get_marka_for_empty(row):
  if (row["param_year"] == None) and (row["param_rok-produkcji"] == None):
    marka = row["breadcrumb"][1]
    model = row["breadcrumb"][2].replace(marka+'-',"")
    return marka
  else:
    return row["my_marka"]

def get_model_for_empty(row):
  if (row["param_year"] == None) and (row["param_rok-produkcji"] == None):
    marka = row["breadcrumb"][1]
    model = row["breadcrumb"][2].replace(marka+'-',"")
    return model
  else:
    return row["my_model"]

all_data["my_marka"] = all_data.apply(get_marka_for_empty, axis=1)
all_data["my_model"] = all_data.apply(get_model_for_empty, axis=1)

list_drogie = ['McLaren', 'Ferrari', 'Maybach', 'Aston Martin', 'Lamborghini',
       'Bentley', 'Tesla', 'Maserati', 'Rolls-Royce', 'Porsche']

def find_drogie_marki(row, list_drogie):
  if row["my_marka"] in list_drogie:
    return True
  return False

all_data["my_drogie"] = all_data.apply(find_drogie_marki, list_drogie=list_drogie, axis=1)

all_data["my_marka"] = all_data["my_marka"].factorize()[0]
all_data["my_model"] = all_data["my_model"].factorize()[0]

In [ ]:
aa = all_data.groupby(["my_marka"])["price_value"].mean()
aa.sort_values(ascending=False).index

Int64Index([ 75,  63,  53,  37,  57,  56,  69,  54,  65,   2,
            ...
             79,  61,  81,  97,  16,  84,  73, 100, 101, 102],
           dtype='int64', name='my_marka', length=103)

lecimy nowe

In [ ]:
del all_data["param_oferta-od"]
all_data["my_handlarz"] = all_data["seller_type"].map(lambda x: True if x == "Dealer" else False)
del all_data["seller_type"]

In [ ]:
def find_uszkodzone(row):
  if (row["param_uszkodzony"]) == "Tak" or (row["param_damaged"] == "Tak"):
    return True
  else:
    return False

all_data["my_uszkodzony"] = all_data.apply(find_uszkodzone, axis=1)

del all_data["param_uszkodzony"]
del all_data["param_damaged"]

In [ ]:
def find_bezwypadkowy(row):
  if (row["param_no-accident"]) == "Tak" or (row["param_bezwypadkowy"] == "Tak"):
    return True
  else:
    return False

all_data["my_bezwypadkowy"] = all_data.apply(find_bezwypadkowy, axis=1)

del all_data["param_no-accident"]
del all_data["param_bezwypadkowy"]

In [ ]:
all_data["my_second_zbior"] = all_data["param_mileage"].map(lambda x: True if x is not None else False)

In [ ]:
del all_data["param_kategoria"] #wszystkie osobowe

In [ ]:
def find_rok_produkcji(row):
  if row["param_rok-produkcji"] is not None:
    return int(row["param_rok-produkcji"])
  elif row["param_year"] is not None:
    return int(row["param_year"])
  else:
    return -1

all_data["my_rok"] = all_data.apply(find_rok_produkcji, axis=1)

del all_data["param_rok-produkcji"]
del all_data["param_year"]

In [ ]:
#for column in all_data.columns.values:
#  if "feature" in column:
#    del all_data[column]

In [ ]:
def find_seller_wojewodztwo(row):
  wojewodztwa = ['Wielkopolskie','Mazowieckie','Śląskie','Małopolskie','Dolnośląskie','Łódzkie','Kujawsko-pomorskie','Lubelskie','Pomorskie','Podkarpackie','Świętokrzyskie','Zachodniopomorskie','Warmińsko-mazurskie','Lubuskie','Śródmieście','Opolskie','Podlaskie']
  if row is not None:
    for jedno in wojewodztwa:
      if jedno in row:
        return jedno
      elif "Warszawa" in row:
        return 'Mazowieckie'
  return -1


all_data["my_woj"] = all_data["seller_address"].map(find_seller_wojewodztwo)  
all_data["my_woj"] = all_data["my_woj"].factorize()[0]

In [ ]:
all_features = all_data.columns.values
all_features = [feat for feat in all_features if "feature_" in feat]

def count_features(row, all_features):
  feats_count = 0
  for feat in all_features:
    if row[feat] is True:
      feats_count += 1
  return feats_count

all_data["my_fe_count"] = all_data.apply(count_features, all_features=all_features, axis=1)

In [ ]:
def find_przebieg(row):
  if row["param_przebieg"] is not None:
    return row["param_przebieg"]
  elif row["param_mileage"] is not None:
    return row["param_mileage"]
  else:
    return -1

all_data["my_przebieg"] = all_data.apply(find_przebieg, axis=1)

del all_data["param_przebieg"]
del all_data["param_mileage"]

In [ ]:
def przebieg_to_int(row):
  try:
    przebieg = int (row.replace(" km","").replace(" ", "") )
  except:
    przebieg = -1

  if przebieg > 2_000_000:
    przebieg = 2_000_000
  return przebieg

all_data["my_przebieg"] = all_data["my_przebieg"].map(przebieg_to_int)

In [ ]:
def log_przebieg(row):
  if row == 0:
    return 0
  elif row == -1:
    return -1
  else:
    return np.log(row)

all_data["my_log_przebieg"] = all_data["my_przebieg"].map(log_przebieg)

In [ ]:
def find_vat(row):
  if (row["param_faktura-vat"] == "Tak") or (row["param_vat-discount"] == "Tak")\
      or (row["param_vat-free"] == "Tak"):
    return True
  else:
    return False

all_data["my_vat1"] = all_data.apply(find_vat, axis=1)

del all_data["param_faktura-vat"]
del all_data["param_vat-free"]
del all_data["param_vat-discount"]

In [ ]:
all_data["my_vat_marza"] = all_data["param_vat-marża"].map(lambda x: True if x == "Tak" else False)
del all_data["param_vat-marża"]

In [ ]:
def find_pojemnosc(row):
  if row["param_pojemność-skokowa"] is not None:
    return row["param_pojemność-skokowa"]
  elif row["param_engine-capacity"] is not None:
    return row["param_engine-capacity"]
  else:
    return -1

all_data["my_pojemnosc"] = all_data.apply(find_pojemnosc, axis=1)

del all_data["param_pojemność-skokowa"]
del all_data["param_engine-capacity"]

In [ ]:
def pojemnosc_to_int(row):
  try:
    pojemnosc = int (row.replace(" cm3","").replace(" ", "") )
  except:
    pojemnosc = -1
  
  if pojemnosc > 10000 and pojemnosc%10 ==0:
    pojemnosc = pojemnosc/10

  if pojemnosc > 10000 and pojemnosc%10 ==0:
    pojemnosc = pojemnosc/10

  if pojemnosc > 10000:
    pojemnosc = -1

  return pojemnosc

all_data["my_pojemnosc"] = all_data["my_pojemnosc"].map(pojemnosc_to_int)

In [ ]:
def rejestracja_polska(row):
  if (row["param_registered-in-poland"] == "Tak") or (row["param_zarejestrowany-w-polsce"] == "Tak"):
    return True
  else:
    return False

all_data["my_rejestracja_polska"] = all_data.apply(rejestracja_polska, axis=1)

del all_data["param_registered-in-poland"]
del all_data["param_zarejestrowany-w-polsce"]

In [ ]:
all_data["my_anglik"] = all_data["param_kierownica-po-prawej-(anglik)"].map(lambda x: True if x == "Tak" else False)
del all_data["param_kierownica-po-prawej-(anglik)"]

In [ ]:
def find_kolor(row):
  if row["param_kolor"] is not None:
    return row["param_kolor"]
  elif row["param_color"] is not None:
    return row["param_color"]
  else:
    return -1

all_data["my_color"] = all_data.apply(find_kolor, axis=1)

del all_data["param_kolor"]
del all_data["param_color"]

all_data["my_color"] = all_data["my_color"].factorize()[0]

In [ ]:
def find_kraj(row):
  if row["param_kraj-pochodzenia"] is not None:
    return row["param_kraj-pochodzenia"]
  elif row["param_country-of-origin"] is not None:
    return row["param_country-of-origin"]
  else:
    return -1

all_data["my_kraj"] = all_data.apply(find_kraj, axis=1)

del all_data["param_kraj-pochodzenia"]
del all_data["param_country-of-origin"]

all_data["my_kraj"] = all_data["my_kraj"].factorize()[0]

In [ ]:
def find_moc(row):
  if row["param_moc"] is not None:
    return row["param_moc"]
  elif row["param_engine-power"] is not None:
    return row["param_engine-power"]
  else:
    return -1

all_data["my_moc"] = all_data.apply(find_moc, axis=1)

del all_data["param_moc"]
del all_data["param_engine-power"]

In [ ]:
def pmoc_to_int(row):
  try:
    moc = int (row.replace(" KM","").replace(" ", "") )
  except:
    moc = -1

  if moc > 1000 and moc%10 ==0:
    moc = moc/10

  if moc > 1000 and moc%10 ==0:
    moc = moc/10

  if moc > 1000:
    moc = -1

  return moc

all_data["my_moc"] = all_data["my_moc"].map(pmoc_to_int)

In [ ]:
def find_skrzynia(row):
  if row["param_skrzynia-biegów"] is not None:
    return row["param_skrzynia-biegów"]
  elif row['param_gearbox'] is not None:
    return row['param_gearbox']
  else:
    return -1

all_data["my_skrzynia"] = all_data.apply(find_skrzynia, axis=1)

del all_data["param_skrzynia-biegów"]
del all_data["param_gearbox"]

all_data["my_automat_skrzynia"] = all_data["my_skrzynia"].map(lambda x: False if x == "Manualna" else True)
all_data["my_skrzynia"] = all_data["my_skrzynia"].factorize()[0]

In [ ]:
def find_paliwo(row):
  if row["param_rodzaj-paliwa"] is not None:
    return row["param_rodzaj-paliwa"]
  elif row['param_fuel-type'] is not None:
    return row['param_fuel-type']
  else:
    return -1

all_data["my_paliwo"] = all_data.apply(find_paliwo, axis=1)

del all_data["param_rodzaj-paliwa"]
del all_data["param_fuel-type"]

all_data["my_paliwo"] = all_data["my_paliwo"].factorize()[0]

In [ ]:
def find_naped(row):
  if row["param_transmission"] is not None:
    return row["param_transmission"]
  elif row['param_napęd'] is not None:
    return row['param_napęd']
  else:
    return -1

all_data["my_naped"] = all_data.apply(find_naped, axis=1)

del all_data["param_transmission"]
del all_data["param_napęd"]

all_data["my_naped_4x4"] = all_data["my_naped"].map(lambda x: "4x4" in str(x))
all_data["my_naped_tyl"] = all_data["my_naped"].map(lambda x: "tylne" in str(x))
all_data["my_naped"] = all_data["my_naped"].factorize()[0]

In [ ]:
def find_drzwi(row):
  if row["param_door-count"] is not None:
    drzwi = int( row["param_door-count"] )
  elif row['param_liczba-drzwi'] is not None:
    drzwi = int( row['param_liczba-drzwi'] )
  else:
    drzwi = -1

  if drzwi > 10:
    drzwi = 4
  return drzwi

all_data["my_drzwi"] = all_data.apply(find_drzwi, axis=1)

del all_data["param_door-count"]
del all_data["param_liczba-drzwi"]

In [ ]:
def find_miejsca(row):
  if row["param_liczba-miejsc"] is not None:
    miejsca = int( row["param_liczba-miejsc"] )
  elif row['param_nr-of-seats'] is not None:
    miejsca = int( row['param_nr-of-seats'] )
  else:
    miejsca = -1

  return miejsca

all_data["my_miejsca"] = all_data.apply(find_miejsca, axis=1)

del all_data["param_liczba-miejsc"]
del all_data["param_nr-of-seats"]

In [ ]:
def find_metalik(row):
  if (row["param_metallic"] is not None) or (row["param_metalik"] is not None):
    return True
  else:
    return False

all_data["my_metalik"] = all_data.apply(find_metalik, axis=1)
del all_data["param_metallic"]
del all_data["param_metalik"]

In [ ]:
def find_typ(row):
  if row["param_typ"] is not None:
    return row["param_typ"]
  elif row['param_body-type'] is not None:
    return row['param_body-type']
  else:
    return -1

all_data["my_typ"] = all_data.apply(find_typ, axis=1)

del all_data["param_typ"]
del all_data["param_body-type"]

all_data["my_typ"] = all_data["my_typ"].factorize()[0]

In [ ]:
def find_rejestracja(row):
  if row["param_pierwsza-rejestracja"] is not None:
    return row["param_pierwsza-rejestracja"]
  elif row["param_first-registration"] is not None:
    return row["param_first-registration"]
  else:
    return -1

all_data["my_rejestracja"] = all_data.apply(find_rejestracja, axis=1)

del all_data["param_pierwsza-rejestracja"]
del all_data["param_first-registration"]

In [ ]:
def get_rejestracja_year(row):  
  if row != -1:
    return int(row[-4:])
  return -1

all_data["my_registration_year"] = all_data["my_rejestracja"].map(get_rejestracja_year)

def get_rejestracja_difference(row):
  if row["my_registration_year"] == -1:
     return -1
  elif row["my_registration_year"] != -1:
    diff = row["my_rok"] - row["my_registration_year"]
    if diff > 100 or diff < -1:
      return -1
    else:
      return diff
  return -1

all_data["my_rej_diff"] = all_data.apply(get_rejestracja_difference, axis=1)
del all_data["my_rejestracja"]

In [ ]:
all_data["my_zabytek"] = all_data["param_zarejestrowany-jako-zabytek"].map(lambda x: True if x == "Tak" else False)
del all_data["param_zarejestrowany-jako-zabytek"]

In [ ]:
import re

def find_first_date(row):
  if row is not None:
    first_year = re.findall("([0-9]{4})-", row)
    if len(first_year) > 0:
      return int(first_year[0])
  return -1

def find_last_date(row):
  if row is not None:
    first_year = re.findall("-([0-9]{4})", row)
    if len(first_year) > 0:
      return int(first_year[0])
    return 2018

all_data["my_first"] = all_data["param_wersja"].map(find_first_date)
all_data["my_last"] = all_data["param_wersja"].map(find_last_date)


def find_difference_first_year(row):
  if row["my_rok"] == -1:
    return -10
  elif row["my_first"] != -1:
    return row["my_rok"] - row["my_first"]

all_data["my_model_age"] = all_data.apply(find_difference_first_year, axis=1)


In [ ]:
def find_wersja(row):
  if row["param_wersja"] is not None:
    return row["param_wersja"]
  elif row['param_version'] is not None:
    return row['param_version']
  else:
    return -1

all_data["my_wersja"] = all_data.apply(find_wersja, axis=1)
all_data["my_wersja"] = all_data["my_wersja"].factorize()[0]
all_data["my_wersja_2"] = all_data["param_wersja"].factorize()[0]

del all_data["param_wersja"]
del all_data["param_version"]

In [ ]:
def find_leasing(row):
  if row["param_leasing"] is not None:
    return row["param_leasing"]
  elif row['param_leasing-concession'] is not None:
    return row['param_leasing-concession']
  else:
    return -1

all_data["my_leasing"] = all_data.apply(find_leasing, axis=1)
all_data["my_leasing"] = all_data["my_leasing"].map(lambda x: True if x == "Tak" else False)


del all_data["param_leasing"]
del all_data["param_leasing-concession"]

In [ ]:
def find_emisja(row):
  if row["param_emisja-co2"] is not None:
    return row["param_emisja-co2"]
  elif row['param_co2-emissions'] is not None:
    return row['param_co2-emissions']
  else:
    return -1

all_data["my_emisja"] = all_data.apply(find_emisja, axis=1)

del all_data["param_emisja-co2"]
del all_data["param_co2-emissions"]

def emisja_to_int(row):
  try:
    emisja = int (row.replace("g/km","").replace(" ", "") )
  except:
    emisja = -1

  if emisja > 1000:
    emisja = -1
  
  return emisja

all_data["my_emisja"] = all_data["my_emisja"].map(emisja_to_int)

In [ ]:
def find_perlowy(row):
  if (row["param_perłowy"] == "Tak") or (row["param_pearl"] == "Tak"):
    return True
  return False

all_data["my_perlowy"] = all_data.apply(find_perlowy, axis=1)

del all_data["param_perłowy"]
del all_data["param_pearl"]

In [ ]:
def find_owner(row):
  if (row["param_pierwszy-właściciel"] == "Tak") or (row["param_original-owner"] == "Tak"):
    return True
  return False

all_data["my_owner"] = all_data.apply(find_owner, axis=1)

del all_data["param_pierwszy-właściciel"]
del all_data["param_original-owner"]

In [ ]:
def find_akryl(row):
  if (row["param_akryl-(niemetalizowany)"] is not None) or (row["param_acrylic"] == "Tak"):
    return True
  return False

all_data["my_akryl"] = all_data.apply(find_akryl, axis=1)

del all_data["param_akryl-(niemetalizowany)"]
del all_data["param_acrylic"]

In [ ]:
def find_filtr(row):
  if (row["param_filtr-cząstek-stałych"] == "Tak") or (row["param_particle-filter"] == "Tak"):
    return True
  return False

all_data["my_filtr"] = all_data.apply(find_filtr, axis=1)

del all_data["param_filtr-cząstek-stałych"]
del all_data["param_particle-filter"]

In [ ]:
all_data["my_tuning"] = all_data["param_tuning"].map(lambda x: True if x == "Tak" else False)
del all_data["param_tuning"]

In [ ]:
def find_opcja(row):
  if (row["param_financing-option"] == "Tak") or (row["param_możliwość-finansowania"] == "Tak"):
    return True
  return False

all_data["my_opcja"] = all_data.apply(find_opcja, axis=1)

del all_data["param_financing-option"]
del all_data["param_możliwość-finansowania"]

In [ ]:
all_data["my_homo"] = all_data["param_homologacja-ciężarowa"].map(lambda x: True if x == "Tak" else False)
del all_data["param_homologacja-ciężarowa"]

In [ ]:
all_data["my_nowe"] = all_data["param_stan"].map(lambda x: True if x == "Nowe" else False)
del all_data["param_stan"]

In [ ]:
def find_aso(row):
  if (row["param_serwisowany-w-aso"] == "Tak") or (row["param_service-record"] == "Tak"):
    return True
  return False

all_data["my_aso"] = all_data.apply(find_aso, axis=1)

del all_data["param_serwisowany-w-aso"]
del all_data["param_service-record"]

In [ ]:
def wykup_to_int(row):
  try:
    wykup = float (row.replace("PLN","").replace(" ", "").replace(",", ".") )
  except:
    wykup = -1
  
  return int(wykup)

all_data["my_wykup"] = all_data['param_wartość-wykupu'].map(wykup_to_int)
del all_data['param_wartość-wykupu']

In [ ]:
def rata_to_int(row):
  try:
    wykup = float (row.replace("PLN","").replace(" ", "").replace(",", ".") )
  except:
    wykup = -1
  
  return int(wykup)

all_data["my_rata"] = all_data['param_miesięczna-rata'].map(rata_to_int)
del all_data['param_miesięczna-rata']

In [ ]:
all_data["my_seller"] = all_data["seller_name"].factorize()[0]
del all_data["seller_name"]

In [ ]:
all_data["my_matowy"] = all_data['param_matowy'].map(lambda x: True if x is not None else False)
del all_data['param_matowy']

In [ ]:
all_data["moje_poz_raty"] = all_data["param_liczba-pozostałych-rat"].map(lambda x: int(float(x.replace(" ","").replace(",","."))) if x is not None else -1)
del all_data["param_liczba-pozostałych-rat"]

zapis

In [ ]:
all_data.to_hdf("drogie-check.h5", "abc")

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'seller_address', 'param_marka-pojazdu', 'param_model',
       'param_engine-code', 'param_vin', 'param_make', 'param_kod-silnika',
       'param_opłata-początkowa', 'param_model-pojazdu',
       'param_monthly-payment-value'],
      dtype='object')]

  encoding=encoding,
